In [9]:
# Importing the libraries 
import pandas as pd
import numpy as np
import pickle
import re
import os
import zipfile
from statsmodels.stats.proportion import proportion_confint
# active labeler related
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import ComplementNB  # corrects for class imbalance, SGD is pretty good too
from sklearn.pipeline import Pipeline
from superintendent import ClassLabeller
from IPython.display import display, Markdown

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),
])
print('done')

done


In [2]:
rootpath = "/hdd/work/d4ad_standardization/"
filepath = "./D4AD_Standardization/data/raw/etpl_all_programsJune3.xls"

columns = [
    "NAME",
    "NAME_1",
    "DESCRIPTION",
    "PREREQUISITES",
    "FEATURESDESCRIPTION",
    "STREET1",
    "CITY",
    "STATE",
    "ZIP",
    "WEBSITE",
    "COUNTY",
    "NONGOVAPPROVAL",
    "STATECOMMENTS",
    "CIPCODE",
    "PROVIDERID",
    "APPROVINGAGENCYID"
]

df = pd.read_excel(rootpath + filepath, usecols=columns)
print('done')

done


In [3]:
# Set up columns to keep, fields, locations for writing
processedpath = "D4AD_Standardization/data/processed/"
externalpath = "D4AD_Standardization/data/external/"

content_is = "NAME_1_lookup_table.csv"

columns_to_save = ['STANDARDIZEDNAME1', 'NAME_1', 'PROVIDERID',
                    'APPROVINGAGENCYID', 'CIPCODE']
print('done')

done


In [10]:
ONET_TOOLS_TECH_URL_NAME = ("https://www.onetcenter.org/dl_files/database/db_20_1_text/Tools%20and%20Technology.txt", "onet_tools_tech.csv")
CAREERONESTOP_CERTIFICATIONS_URL_NAME = ("https://www.careeronestop.org/TridionMultimedia/tcm24-48614_CareerOnestop_Certifications_07072020.zip", "career_one_stop.zip")

filepath = rootpath + externalpath

for dataset in (ONET_TOOLS_TECH_URL_NAME, CAREERONESTOP_CERTIFICATIONS_URL_NAME):
    url, filename = dataset
    print("running ...", f'\nwget -O {filepath+filename} {url}')
    os.system(f'wget -O {filepath+filename} {url}')
    print("filetype is",  filename[-3:])

    if filename[-3:] == 'zip':
        with zipfile.ZipFile(filepath+filename,"r") as zip_ref:
            zipdir = filepath+filename[:-4]
            print("unzipping {} to ...".format(filename), zipdir)
            os.mkdir(zipdir)
            zip_ref.extractall(zipdir)

KeyboardInterrupt: 

In [5]:
pd.set_option('display.max_rows', False)

the_df = df #df.sample(n=10000, random_state=42)

In [6]:
# A) 
# The program or course name can start or end with a matching parenthesis. In these cases
# we assume that no other matching parenthesis are present and apply 
# an appropriate regex for that...

# First, set up standardized column with default values
the_df["STANDARDIZEDNAME_1"] = ""

# ... then extract names for those with opening parens
open_parenthesis_index = the_df.NAME_1.str[0] == '('
open_parenthesis_regex = '''
                (?P<paren>\(.*\)) # get the first parathesis
                (?P<the_name>.*)  # then get the actual name
                '''

the_df.loc[open_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[open_parenthesis_index, 'NAME_1']\
          .str\
          .extract(open_parenthesis_regex, flags=re.VERBOSE).the_name

# ... then extract names for those with closing parens
close_parenthesis_index = the_df.NAME_1.str[-1] == ')'
closing_parenthesis_regex = '''
                (?P<the_name>.*)  # get the actual name
                (?P<paren>\(.*\)) # get the last parathensis                
                '''
the_df.loc[close_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[close_parenthesis_index, 'NAME_1']\
          .str\
          .extract(closing_parenthesis_regex, flags=re.VERBOSE).the_name

# ... then we copy over content that has a internal parenthesis with those
# parenthesis removed and ignore everything after, e.g. "ABC (123) DEF" --> "ABC"
internal_parenthesis_index =\
    the_df['NAME_1'].str.contains('\(|\)', regex=True) &\
        ~(close_parenthesis_index|open_parenthesis_index)

the_df.loc[internal_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[internal_parenthesis_index, 'NAME_1']\
          .str\
          .extract(closing_parenthesis_regex, flags=re.VERBOSE).the_name

# ... finally, just copy over everything else
no_parenthesis_index = ~(close_parenthesis_index |\
                         open_parenthesis_index  |\
                         internal_parenthesis_index)
the_df.loc[no_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[no_parenthesis_index, 'NAME_1']

print('done')

done


In [7]:
# 2)
# So now we have silver version data of program, course names
# from the cell above, in STANDARDIZEDNAME_1
#
# To make an incrementally better version we need to expand 
# abbreviations and acroynmns.

# Here I identify two pretty common cases of acronymns and abbreviations:
#   All caps
#   Xx*. <- capitalized inital letter ending with a period

# Now let's attempt to extract presumed acronyms and see if we can
# directly label them. I assume there are far fewer unique abbreviations
# so that a person can actually do this in a short amount of time
flags = re.VERBOSE

#  TODO: check if abbreviation labeled file already exists, if it does
# we skip this portion

# Pandas/Python doesn't like this verbose regex but likes other?
# all_caps_regex = '''
#                 \b(?P<all_caps>[A-Z]+)  # Get all caps words
#                 [\s,:\d]                # sit before a space, comma or digit
#                 '''
all_caps_regex = r'\b(?P<all_caps>[A-Z]+)[\s,:\d]'

dotted_word_regex = r'(?P<dot_abbreviation>[A-Z][a-z]+\.)'
dotted_word_regex =\
    """
    (?P<dot_abbreviation>[a-zA-Z][a-z]+\.)
    """

the_regexs = "|".join([all_caps_regex, dotted_word_regex])

the_abbreviations =\
    the_df['STANDARDIZEDNAME_1'].str\
                                .extractall(
                                    pat=the_regexs,
                                    flags=flags)
print('done')

done


In [15]:
# Since we've run on the entire dataset we can now
# flatten the dataframe, de-duplicate and then directly label

len(the_abbreviations.all_caps.unique()) +\
    len(the_abbreviations.dot_abbreviation.unique()) # 1151

# now we need to get the count of unique abbreviations so that we can
# label in priority order. We also drop those abbreviations only occuring once
# since they have a 1 / 26,660 chance of occuring (not worth our effort)

# to properly label the all caps and abbreviations we need the 
# context in which they occur. Since we're mapping to one definition
# we assume only the first instance is really needed and label off of that
abbreviations_to_label =\
    pd.concat(
        (the_abbreviations.drop_duplicates(
            subset=['all_caps'],
            keep='first')['all_caps'],
         the_abbreviations.drop_duplicates(
             subset=['dot_abbreviation'],
             keep='first')['dot_abbreviation']
        ),
        axis=0
    ).dropna()\
     .droplevel('match')\
     .reset_index() # so that index is a column

abbreviations_to_label.rename(columns={'index':'the_df_index', 0:'abbreviation'}, inplace=True)
print('done')

done


In [45]:
# we also filter out thosea abbreviations already labeled in a prior
# session

# Note: this is very dependent on the ordering of the source ETLP datafile
rootpath = "/hdd/work/d4ad_standardization/"
interimpath = "D4AD_Standardization/data/interim/"
abbreviation_pickle = rootpath + interimpath + 'abbreviation_label.pickle'

if os.path.exists(abbreviation_pickle):
    expanded_labels = pd.read_pickle(abbreviation_pickle)
    last_labeled_index = expanded_labels.index(None)
    former_labels = abbreviations_to_label.abbreviation[:last_labeled_index] #set(already_labeled[:last_labeled_index])

unseen_abbreviations =\
    abbreviations_to_label.query('abbreviation not in @former_labels')
unseen_abbreviations.abbreviation.value_counts()
# note I'm seeing 1 across the board, both when using not in and in
#   which suggests that we leave these be for now because their occurence
# is so rare out of 26,660, although the bulk may be significant; better
# to circle back though


ACTIONSCRIPT     1
Impl.            1
Dev.             1
SERVICES         1
LSAT             1
AWARENESS        1
AFOT             1
UCC              1
CPS              1
Ft.              1
PREVENTION       1
STANDARDIZED     1
                ..
SQF              1
Salesforce.      1
DEVELOPMENTAL    1
AWCA             1
Secr.            1
OHSA             1
MCTS             1
CQT              1
CLS              1
WIOA             1
CORPORATE        1
NFPA             1
Name: abbreviation, Length: 845, dtype: int64

In [132]:
#  So now we manually label them and dump them here
#  This is the procedure we follow
#       A) if a capitalized word is an entire word, leave it alone (no label)
#       B) provide a label for all dotted abbreviated words
def display_func(row):
    # Note: We use globally available the_df to get context, bad form I know
    display(
        Markdown(
            "**Context:** " +  the_df.loc[ row.the_df_index ].NAME_1 \
        +   "\n\n" + row.abbreviation
        )
    )

def preprocessor(x, y):
    # only take standardized column, leave everything else
    return x.abbreviation, y

labelling_widget = ClassLabeller(
    features=abbreviations_to_label,
    model=pipeline,
    model_preprocess=preprocessor,
    display_func=display_func,
    options=['No Label'],
    acquisition_function='entropy'
)

labelling_widget


ClassLabeller(children=(HBox(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.…

In [145]:
#  Every now and then, with the labels in had we simply output them (if a file doesn't already exist)
# so that we can save them incrementally. We should manually rename older files; this should
# be basically a 1 time process.

# Temp, save work locally so we don't loooooose it! 
pickle.dump(labelling_widget.new_labels,
            open(abbreviation_pickle, 'wb'))
print('done')

#sum([1 for label in labelling_widget.new_labels if label != None])

done


In [68]:
# Now we do a mass search and replace on STANDARDIZED_NAME_1 and STANDRADIZED_NAME with the labels that we have

# We follow this overflow thread
# see: https://stackoverflow.com/a/48887382/3662899

# First, construct an label to abbreviation dictionary
label_mapper =\
    pd.DataFrame(
        {
            "abbreviation": \
abbreviations_to_label.abbreviation[: last_labeled_index].values,
            "expanded": expanded_labels[:last_labeled_index]
        }
    )
copy_over_index = (label_mapper.expanded == "No Label") | (label_mapper.expanded == "Submit.")
label_mapper.expanded[copy_over_index] =\
    label_mapper.abbreviation[copy_over_index]
rep_dict = dict(zip(label_mapper.abbreviation, label_mapper.expanded))

pattern = re.compile("|".join([re.escape(k) for k in rep_dict.keys()]), re.M)

def multiple_replace(string):    
    return pattern.sub(lambda x: rep_dict[x.group(0)], string)

#the_df['MULTI_REPLACE_STANDARDIZEDNAME_1'] =\
#    the_df['STANDARDIZEDNAME_1'].map(multiple_replace)

#the_df.loc[['MULTI_REPLACE_STANDARDIZEDNAME_1', 'STANDARDIZEDNAME_1']]
#the_df[['STANDARDIZEDNAME_1', 'NAME_1']]


,STANDARDIZEDNAME_1,NAME_1
0,Automated Office Systems Processor,Automated Office Systems Processor
1,Bus.Soft. App/Office Proc.Legal/Mach.Transcrip...,Bus.Soft. App/Office Proc.Legal/Mach.Transcrip...
2,Bus.Soft. App./Comp.Office Proc.Medical,Bus.Soft. App./Comp.Office Proc.Medical
3,Microsoft Office Specialist II,Microsoft Office Specialist II
4,Computerized Financial Accounting,Computerized Financial Accounting
5,Introduction to Computers,Introduction to Computers
6,"HVAC Principals I, III, Math - Certificate w/MCCC","HVAC Principals I, III, Math - Certificate w/MCCC"
7,Cosmetology,Cosmetology
8,Diesel Mechanics Technology/Technician,Diesel Mechanics Technology/Technician
...,...,...


In [10]:

# Here we ingest Career One Stop certifications
#   I was goign to use this to de-acroymn-ize mentions but now am unsure
# if this is critical.

# Here we read in a .sql directly as text and parse out the data.
# I do this to avoid the need for a database, db drivers, etc. 
# That said, this represented some investment in constructing the right regexs
path = rootpath + externalpath + 'career_one_stop/'
credential_sql = 'TEST-2-CERTIFICATIONS.sql' # '2-CERTIFICATIONS.sql'

with open(path + credential_sql) as sql:
    my_string = sql.read()

header_names =\
    (
        'CERT_ID', 'CERT_NAME', 'ORG_ID', 'TRAINING', 'EXPERIENCE', 
        'EITHER', 'EXAM', 'RENEWAL', 'CEU', 'REEXAM', 
        'CPD', 'CERT_ANY', 'URL', 'ACRONYM', 'NSSB_URL', 
        'CERT_URL', 'CERT_LAST_UPDATE', 'KEYWORD1', 'KEYWORD2', 'KEYWORD3', 
        'SUPPRESS', 'DATEADDED', 'COMMENTS', 'VERIFIED', 'UPDATEDBY', 
        'CERT_DESCRIPTION', 'DELETED', 'EXAM_DETAILS'
    )

# Pandas assumes atomic python types when reading from records,
# See: https://github.com/pandas-dev/pandas/issues/9381, so we need to use
# Python types here
dtypes =\
    np.dtype(
        "str, str, float, float,"
        "float, float, float, str,"
        "float, float, float, float,"
        "str, str, str, str,"
        "str, str, str, str,"
        "str, str, str, str," 
        "str, str, float, str"
    )

flags = re.MULTILINE | re.DOTALL | re.VERBOSE
the_fields_regex =\
    """
    (?P<values>Values\n\s+\()  # Start with the word Value <newline> (
        (?P<fields>.*?)        #    Grab all the field content
    (?P<end>\);)               # ... which stops at the terminating paren, ;
    """

the_fields = re.compile(the_fields_regex, flags=flags)

a_field_regex =\
    """
    '(?P<string>.*?)'[,)]           # get a quoted string ending at comma or paran or
    |(?P<date_time>TO_DATE\(.*?\))  # get the TO_DATE, parse out actual date later or
    |(?P<num>\d),                   # get numeric or
    |(?P<null>NULL)                 # get NULL
    """

a_field = re.compile(a_field_regex, flags=flags)

require_field_numbers = [1] # should be 13

def yield_certification_records(sql_file=my_string, require_field_numbers=require_field_numbers):
    # do we skip those w/o certain fields, like acronymns
    temp_data = [0]*28
    for match in the_fields.finditer(sql_file):
        break_match = False

        for index, field in enumerate(a_field.finditer( match.group('fields') )):
            grp = None
            for grp, value in field.groupdict().items():
                if value:
                    # then we transform the string value into the appropriate type, given the group name
                    if grp == 'date_time':
                        #  There is a difference between https://regex101.com/r/yphUXY/1/
                        # and what I see Python do here; if I don't capture the entire thing
                        # it gets re-raised as another potential match, even if I use ?:, etc.
                        value = value[9:28] # todo: convert to datetime
                    if grp == 'null':
                        value = None
                        if index in require_field_numbers:
                            break_match = True

                    if grp == 'num':
                        value = int(value)

                    temp_data[index] = value
                    break # only one possible match value
            if break_match: # and don't look at other fields
                break

        if not break_match:
            yield tuple(value for value in temp_data)
        
        break_match = False

certification_df =\
    pd.DataFrame.from_records(
        yield_certification_records(),
        columns=header_names)
certification_df

,CERT_ID,CERT_NAME,ORG_ID,TRAINING,EXPERIENCE,EITHER,EXAM,RENEWAL,CEU,REEXAM,...,KEYWORD2,KEYWORD3,SUPPRESS,DATEADDED,COMMENTS,VERIFIED,UPDATEDBY,CERT_DESCRIPTION,DELETED,EXAM_DETAILS
0,10249,SAP Certified Technology Associate - System Ad...,0106,NaN,NaN,None,1,,None,None,...,None,None,1,01/21/2015 16:31:10,None,0,28,This certification path will validate your cap...,1,None
1,10052,IBM Certified BPM Developer - WebSphere Lombar...,0814,1.0,1.0,None,1,,None,None,...,None,None,1,10/21/2014 10:43:08,None,0,28,This intermediate level certification is inten...,1,None
2,10096,IBM Certified Specialist - Systems Networking ...,0814,0.0,0.0,None,1,,None,None,...,None,None,1,10/24/2014 14:24:09,None,0,28,The IBM Certified Specialist - System Networki...,1,None


In [ ]:
# 3) 
# Then go after odd static patterns that are common 
# ... A.A., AAS,e ends-with "/", etc etc
# "Applied Certificate in..." <--- thing is, this could really be a program
# the_df.STANDARDIZEDNAME_1 =\
#     the_df.STANDARDIZEDNAME_1.str.replace("A.A.","", case=False)




In [ ]:
# 4)
# Now we write out this  

In [6]:
# This is the evaluation part of the program and course name standardizations

def display_func(row):
    """
    The display function gets passed your data - in the
    case of a dataframe, it gets passed a row - and then
    has to "display" your data in whatever way you want.

    It doesn't need to return anything
    """
    display(Markdown(row["STANDARDIZEDNAME_1"]))
    #display(Markdown("**At:** " + row["timestamp"]))

def preprocessor(x, y):
    # only take standardized column, leave everything else
    return x["STANDARDIZEDNAME_1"], y

labelling_widget = ClassLabeller(
    features=the_df,
    model=pipeline,
    model_preprocess=preprocessor,
    display_func=display_func,
    options=['standardized', 'not standardized'],
    acquisition_function='margin'
)

labelling_widget

ClassLabeller(children=(HBox(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.…

In [26]:
# insert bionomial proprtion esimator here

def print_CI(labels, response_is_standardized = "standardized", method = "jeffreys"):
    successful_count = sum(
        response_is_standardized == label for label in labels
    )
    not_examined_count = sum(
        None == label for label in labels
    )

    CI = proportion_confint(
            count= successful_count,
            nobs= len(labels) - not_examined_count,
            alpha = 0.95,
            method=method
        )
    print(f"{method} bionomial proportion is: [{CI[0]:.2f}, {CI[1]:.2f}]",
)
print_CI(labels=labelling_widget.new_labels)

jeffreys bionomial proportion is: [0.88, 0.88]


In [142]:
# ... finally we can write this out as our first complete lookup table
# for the NAME field
the_df.to_csv(rootpath + processedpath + "{}".format(content_is),
              index = False,
              chunksize = 10000,
              columns=columns_to_save)